In [ ]:
import os
import pandas as pd
import cx_Oracle
from dotenv import load_dotenv

### Inicializálás

In [ ]:
load_dotenv()
username = os.getenv('ORACLE_USERNAME')
password = os.getenv('ORACLE_PASSWORD')
dsn = os.getenv('ORACLE_DSN')
pool = cx_Oracle.SessionPool(user=username, password=password, dsn=dsn, min=2, max=5, increment=1)

input_file = 'BETÖLTŐ_május.xlsx'
output_file = 'elokeszit.xlsx'
KID='397451/2024'
AHT='397451'

def get_data(ksh):
    query = f'''
select 
    onk.ksh, 
    onk.onev, 
    SUBSTR(st.torzs_sz||'-'||st.afakod||'-'||st.mk, 0, 8) as torzsszam, 
    cim.irszam, cim.koztnev||' '||cim.kozjell as cim, 
    cim.hsz1 as hsz, 
    TRIM(rk.megnevezes) as regio
from SUPPORTSYS.t_onkorm_tech onk
left join (select * from SUPPORTSYS.t_stat) st on st.ksh=onk.ksh
left join (select * from SUPPORTSYS.t_cim) cim on cim.ksh=onk.ksh
left join SUPPORTSYS.t_regio rk on rk.regiokod=onk.rk
where onk.ksh='{ksh}'
'''
    connection = pool.acquire()
    cursor = connection.cursor()
    cursor.execute(query)
    data = cursor.fetchall()    
    pool.release(connection)
    return data

### EBR betöltő beolvasás

In [ ]:
df = pd.read_excel(f'./data/{input_file}', usecols=[1, 2, 6], names=['KSH', 'ONK', 'AMOUNT'])
df['KSH'] = df['KSH'].astype(str).str.zfill(7)
df['KID'] = KID
df['AHT'] = AHT
df['ONEV'] = ''
df['TORZSSZAM'] = ''
df['IRSZAM'] = ''
df['CIM'] = ''
df['HSZ'] = ''
df['REGIO'] = ''

df.head()

### Szükséges adatok az adatbázisból

In [ ]:
column_names = ['ONEV', 'TORZSSZAM', 'IRSZAM', 'CIM', 'HSZ', 'REGIO']

for index, row in df.iterrows():
    ksh_value = row['KSH']
    data = get_data(ksh_value)
    for i, column_name in enumerate(column_names):
        df.at[index, column_name] = data[0][i+1]
df.head()

### Bezárás

In [ ]:
pool.close()
df.to_excel(f'./data/{output_file}', index=False)